In [ ]:
import os
import sys
from google.colab import drive
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/ICBF/data


# **Cruce entre tablas de datos sociodemográficos y nutricionales**

####**Descripción:**

Asigna correctamente los identificadores `IdBeneficiario` e `Id` a cada registro de la tabla de datos nutricionales, a partir de la tabla de datos sociodemográficos, corrigiendo algunos duplicados y algunos datos faltantes, y asignando un nuevo valor de `IdBeneficiario` a cada registro que no lo tenga. 

Genera una partición aleatoria de los beneficiarios por su identificador `IdBeneficiario` en un conjunto de entrenamiento y uno de prueba (con el 80% y el 20% de los registros, respectivamente) para el modelado.
 
####**Requerimientos:**

- Tabla de datos sociodemográficos preprocesada `preprocessed_data/sociodemo.parquet`.
- Tabla de datos de los beneficiarios con tomas nutricionales preprocesada `preprocessed_data/tomas_beneficiario.parquet`. 
- Tabla de datos `preprocessed_data/tomas_normalizado.parquet` con datos nutricionales de cada toma.

####**Salidas:**

- Tabla de relación `auxiliary_data/IdBeneficiario_Id.parquet` que relaciona los identificadores de las tablas de datos nutricionales (`IdBeneficiario`) y sociodemográficos (`Id`).
- Tabla de relación `auxiliary_data/ids_Registro.parquet` que relaciona cada registro de las tablas de datos nutricionales con los identificadores de  `IdBeneficiario` e `Id` que le corresponden.
- Conjuntos serializados `auxiliary_data/IdBeneficiario_train.pickle` y `auxiliary_data/IdBeneficiario_test.pickle` de los identificadores `IdBeneficiario` de los beneficiarios de los conjuntos de entrenamiento y prueba, respectivamente.
- Tabla de datos `clean_data/sociodemo_ids.parquet` con datos sociodemográficos filtrada para incluir solo los beneficiarios que tienen tomas nutricionales, con los identificadores ya corregidos.
- Tabla de datos `clean_data/tomas_beneficiario_ids.parquet` de los beneficiarios con tomas nutricionales preprocesada, con los identificadores ya corregidos.
- Tabla de datos `clean_data/tomas_normalizado_ids.parquet` con datos nutricionales de cada toma preprocesada, con los identificadores ya corregidos.

In [ ]:
# Librerias relevantes
import time
import pandas as pd
import numpy as np
import json
import joblib
from sklearn.model_selection import train_test_split
pd.set_option('mode.chained_assignment', None)

### **Lectura de los archivos**

In [ ]:
t0 = time.time()
# Columnas de identificadores útiles
cols_id = ['IdBeneficiario', 'Id']
# Columnas auxiliares propias de cada beneficiario
cols_aux = ['FechaNacimiento', 'Sexo']
# Lee las columnas relevantes de las tablas de datos nutricionales y sociodemograficos preprocesadas
tom = pd.read_parquet('preprocessed_data/tomas_beneficiario.parquet')
tom = tom[['Registro'] + cols_id + cols_aux]
soc = pd.read_parquet('preprocessed_data/sociodemo.parquet')
soc = soc[cols_id + cols_aux]

tom.info()
soc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6007561 entries, 0 to 6007560
Data columns (total 5 columns):
 #   Column           Dtype         
---  ------           -----         
 0   Registro         UInt32        
 1   IdBeneficiario   UInt32        
 2   Id               UInt32        
 3   FechaNacimiento  datetime64[ns]
 4   Sexo             category      
dtypes: UInt32(3), category(1), datetime64[ns](1)
memory usage: 137.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5945350 entries, 0 to 5945349
Data columns (total 4 columns):
 #   Column           Dtype         
---  ------           -----         
 0   IdBeneficiario   UInt32        
 1   Id               UInt32        
 2   FechaNacimiento  datetime64[ns]
 3   Sexo             float64       
dtypes: UInt32(2), datetime64[ns](1), float64(1)
memory usage: 147.4 MB


### **Exploración preliminar**

In [ ]:
# Identificadores faltantes en cada tabla
print('Identificadores faltantes en tabla de datos nutricionales')
print(tom[cols_id].isna().sum())
print('Identificadores faltantes en tabla de datos sociodemográficos')
print(soc[cols_id].isna().sum())

Identificadores faltantes en tabla de datos nutricionales
IdBeneficiario        977
Id                3188152
dtype: int64
Identificadores faltantes en tabla de datos sociodemográficos
IdBeneficiario    4436351
Id                      0
dtype: int64


In [ ]:
# Identificadores duplicados en la tabla de datos sociodemográficos
soc[cols_id].duplicated(subset=['Id']).sum(), \
soc[cols_id].dropna(subset=['IdBeneficiario']).duplicated(subset=['IdBeneficiario']).sum()

(0, 0)

In [ ]:
# Identificadores 'Id' que existen solo en la tabla de datos nutricionales,
# pero no en la tabla de datos sociodemográficos
Ids = set(soc['Id'])
Id_inexistentes = tom[tom['Id'].notna() & ~tom['Id'].isin(Ids)]['Id']
len(Id_inexistentes)

80

In [ ]:
# Debe existir una correspondecia biunívoca entre ambos conjuntos de identificadores:
# 'IdBeneficiario' e 'Id'
correspondencia_ids = tom[['IdBeneficiario', 'Id']].dropna().drop_duplicates()
# Identificadores de una clase con más de una correspondencia en la otra clase
correspondencia_ids.duplicated(subset=['IdBeneficiario']).sum(), \
correspondencia_ids.duplicated(subset=['Id']).sum()

(3, 431)

En la tabla de datos nutricionales existen 80 identificadores `Id` que no existen en la tabla de datos sociodemográficos.
Además, un mismo valor de `IdBeneficiario` se corresponde con varios `Id` un total de 3 veces. El caso inverso (un mismo `Id` asignado a diferentes valores de `IdBeneficiario`) ocurre 431 veces.

Dado que no existen identificadores duplicados en la tabla de datos sociodemográficos, se usarán las correspondencias en esta tabla para corregir los identificadores inconsistentemente duplicados de la tabla de datos nutricionales.

### **Eliminación de valores de `Id` inexistentes en la base de datos sociodemográficos**

In [ ]:
# Imputar NaNs a los 'Id' no presentes en la tabla de datos sociodemográficos
reemplazo = dict()
for Id in Id_inexistentes:
  reemplazo[Id] = np.NaN
tom['Id'] = tom['Id'].replace(reemplazo).astype('UInt32')

In [ ]:
# Identificadores 'Id' que existen solo en la tabla de datos nutricionales,
# pero no en la tabla de datos sociodemográficos
Ids = set(soc['Id'])
Id_inexistentes = tom[tom['Id'].notna() & ~tom['Id'].isin(Ids)]['Id']
len(Id_inexistentes)

0

In [ ]:
# Debe existir una correspondecia biunívoca entre ambos conjuntos de identificadores:
# 'IdBeneficiario' e 'Id'
correspondencia_ids = tom[['IdBeneficiario', 'Id']].dropna().drop_duplicates()
correspondencia_ids.duplicated(subset=['IdBeneficiario']).sum(), \
correspondencia_ids.duplicated(subset=['Id']).sum()

(1, 430)

###**Asignación de `IdBeneficiario` a partir de la tabla de datos sociodemográficos**

In [ ]:
# Se respeta la correspondencia 'Id' --> 'IdBeneficiario' de la tabla de datos sociodemográficos
IdB_corregido = tom[['Id']].merge(soc[cols_id], on='Id', how='left')['IdBeneficiario'].\
  fillna(tom['IdBeneficiario'])
tom['IdBeneficiario'] = IdB_corregido

In [ ]:
# Debe existir una correspondecia biunívoca entre ambos conjuntos de identificadores:
# 'IdBeneficiario' e 'Id'
correspondencia_ids = tom[['IdBeneficiario', 'Id']].dropna().drop_duplicates()
correspondencia_ids.duplicated(subset=['IdBeneficiario']).sum(), \
correspondencia_ids.duplicated(subset=['Id']).sum()

(0, 0)

In [ ]:
# Identificadores faltantes en tabla de datos nutricionales
print('Identificadores faltantes en tabla de datos nutricionales')
print(tom[cols_id].isna().sum())

Identificadores faltantes en tabla de datos nutricionales
IdBeneficiario        905
Id                3188232
dtype: int64


La contrastación de los identificadores con la tabla de datos sociodemográficos resuelve todas las duplicaciones inconsistentes de `IdBeneficiario` y `Id` y algunos valores de `IdBeneficiario` faltantes en la tabla de datos nutricionales. Todas las correspondencias inconsistentes han sido corregidas.

### **`Id` presente e `IdBeneficiario` nulo**

In [ ]:
# Registros con identificadores nulos
IdB_nulo = tom['IdBeneficiario'].isna()
Id_nulo = tom['Id'].isna()
# Registros que presentan solo el identificador `Id`
solo_Id = tom.loc[IdB_nulo & ~Id_nulo]
solo_Id

Registro  IdBeneficiario       Id FechaNacimiento Sexo
1656826   1656826            <NA>  1623978             NaT  NaN
1656828   1656828            <NA>  2575334             NaT  NaN
1656831   1656831            <NA>  4378889             NaT  NaN
1669692   1669692            <NA>  5617890             NaT  NaN
1669696   1669696            <NA>  5781126             NaT  NaN
1669698   1669698            <NA>  4677938             NaT  NaN
1669699   1669699            <NA>  3200546             NaT  NaN
1669710   1669710            <NA>  3367057             NaT  NaN
1669711   1669711            <NA>  5811956             NaT  NaN
1676116   1676116            <NA>  5121792             NaT  NaN
1676127   1676127            <NA>  5211844             NaT  NaN
1676136   1676136            <NA>  3328051             NaT  NaN

In [ ]:
soc[soc.Id.isin(solo_Id['Id'])]

IdBeneficiario       Id FechaNacimiento  Sexo
1623950            <NA>  1623978      2000-11-15   0.0
2575286            <NA>  2575334      2009-10-30   0.0
3200494            <NA>  3200546      2015-07-20   1.0
3327996            <NA>  3328051      2016-07-17   0.0
3367002            <NA>  3367057      2013-02-17   1.0
4378817            <NA>  4378889      2016-05-26   0.0
4677860            <NA>  4677938      2017-09-15   1.0
5121708            <NA>  5121792      2017-03-04   0.0
5211755            <NA>  5211844      2017-01-26   1.0
5617793            <NA>  5617890      2017-01-28   0.0
5781027            <NA>  5781126      2017-06-13   0.0
5811857            <NA>  5811956      2015-11-28   1.0

Existen 12 individuos que tienen asignado un `Id` pero no un `IdBeneficiario`, cada uno con una toma nutricional; esto es cierto en ambas tablas.

### **`IdBeneficiario` nulo**

In [ ]:
IdB_nulo = tom['IdBeneficiario'].isna()
# Registros que aún no presentan el identificador `IdB`
sin_IdB = tom.loc[IdB_nulo]
sin_IdB

Registro  IdBeneficiario    Id FechaNacimiento Sexo
1650539   1650539            <NA>  <NA>             NaT  NaN
1650540   1650540            <NA>  <NA>             NaT  NaN
1650541   1650541            <NA>  <NA>             NaT  NaN
1650542   1650542            <NA>  <NA>             NaT  NaN
1650543   1650543            <NA>  <NA>             NaT  NaN
...           ...             ...   ...             ...  ...
1741417   1741417            <NA>  <NA>             NaT  NaN
1741418   1741418            <NA>  <NA>             NaT  NaN
1741419   1741419            <NA>  <NA>             NaT  NaN
1741420   1741420            <NA>  <NA>             NaT  NaN
1741421   1741421            <NA>  <NA>             NaT  NaN

[905 rows x 5 columns]

Existen 905 registros sin `IdBeneficiario` asignado (incluyendo los 12 anteriores). Se le asignará a cada uno un `IdBeneficiario` nuevo.

### **Asignación de nuevos valores de `IdBeneficiario`**

In [ ]:
# Crea nuevos valores de 'IdBeneficiario' a partir del valor máximo actual
nuevos_idB = [tom['IdBeneficiario'].max() + i + 1 for i in range(IdB_nulo.sum())]
tom.loc[IdB_nulo, 'IdBeneficiario'] = nuevos_idB
print(f"'IdBeneficiario' {IdB_nulo.sum()} nuevos identificadores creados:",
      f"{time.time() - t0:.2f} s.")

'IdBeneficiario' 905 nuevos identificadores creados: 75.10 s.


In [ ]:
# Registros sin 'IdBeneficiario'
print(f"{tom['IdBeneficiario'].isna().sum()} registros sin 'IdBeneficiario'.")
# Debe existir una correspondecia biunívoca entre ambos conjuntos de identificadores:
# 'IdBeneficiario' e 'Id'
correspondencia_ids = tom[['IdBeneficiario', 'Id']].dropna().drop_duplicates()
correspondencia_ids.duplicated(subset=['IdBeneficiario']).sum(), \
correspondencia_ids.duplicated(subset=['Id']).sum()

0 registros sin 'IdBeneficiario'.


(0, 0)

In [ ]:
# Identificadores faltantes en tabla de datos nutricionales
print('Identificadores faltantes en tabla de datos nutricionales')
print(tom[cols_id].isna().sum())

Identificadores faltantes en tabla de datos nutricionales
IdBeneficiario          0
Id                3188232
dtype: int64


In [ ]:
# Asegura que las correspondencias entre 'IdBeneficiario' e 'Id' estén correctas
tom['Id'] = tom.drop(columns=['Id']).\
  merge(correspondencia_ids, on='IdBeneficiario', how='left')['Id']

len(tom[['IdBeneficiario', 'Id']].dropna().drop_duplicates()), \
len(correspondencia_ids)

(1508837, 1508837)

Todos los registros de la tabla de datos nutricional tienen un identificador `IdBeneficiario` y no existen inconsistencias. Existen ~1.5 millones de beneficiarios presentes en ambas tablas de datos.

### **Partición del conjunto de beneficiarios en conjuntos de entrenamiento y prueba**
El conjunto de beneficiarios actual se dividirá aleatoriamente en conjuntos de entrenamiento (80%) y prueba (20%).

Como la partición es completamente aleatoria, ningún procesamiento ni filtrado posterior debe afectar significativamente dicha proporción 80% / 20%. 

In [ ]:
# Conjunto de beneficiarios
IdBeneficiario = tom['IdBeneficiario'].drop_duplicates().sort_values()
# Partición 80% training / 20% testing
IdB_train, IdB_test = train_test_split(IdBeneficiario, test_size=0.2, random_state=12)
IdB_train, IdB_test = set(IdB_train), set(IdB_test)
n_beneficiarios, n_train, n_test = len(IdBeneficiario), len(IdB_train), len(IdB_test)
print(f"{n_beneficiarios} beneficiarios en la tabla de datos nutricionales.")
print(f"{n_train} ({n_train / n_beneficiarios:.0%}) beneficiarios en el conjunto de entrenamiento.")
print(f"{n_test} ({n_test / n_beneficiarios:.0%}) beneficiarios en el conjunto de prueba.")

3362573 beneficiarios en la tabla de datos nutricionales.
2690058 (80%) beneficiarios en el conjunto de entrenamiento.
672515 (20%) beneficiarios en el conjunto de prueba.


### **Guardado**

In [ ]:
# Tablas de relación entre identificadores
correspondencia_ids.to_parquet('auxiliary_data/IdBeneficiario_Id.parquet')
tom[['Registro'] + cols_id].to_parquet('auxiliary_data/ids_Registro.parquet')
# Partición de beneficiarios en conjuntos de entrenamiento y prueba
joblib.dump(IdB_train, 'auxiliary_data/IdBeneficiario_train.pickle')
joblib.dump(IdB_test, 'auxiliary_data/IdBeneficiario_test.pickle')

print(f"Archivos auxiliares guardados: {time.time() - t0:.2f} s.")

# Aplica las correcciones a los archivos preprocesados
for dataset in os.listdir('preprocessed_data'):
  name = dataset[:-8]
  data = pd.read_parquet(f'preprocessed_data/{name}.parquet')
  columns = data.columns
  # Para las tablas de datos nutricionales utiliza el número de 'Registro' (fila en la tabla original)
  # para asignar los identificadores 'Id' e 'IdBeneficiario' correctamente
  if dataset[:5] == 'tomas':
    data = data.drop(columns=cols_id).merge(tom[['Registro'] + cols_id], on='Registro', how='left')
  # Para la tabla de datos sociodemográfico utiliza el 'IdBeneficiario' para asignar el 'Id' correctamente
  elif dataset[:5] == 'socio':
    data = data.drop(columns=['IdBeneficiario']).merge(correspondencia_ids, on='Id', how='left')
    # Elimina los registros de beneficiarios que no están presentes en la tabla de datos nutricionales
    data = data.dropna(subset = ['IdBeneficiario'])
  # Guarda el nuevo archivo con los identificadores correctos con el sufijo '_ids'
  data.to_parquet(f'clean_data/{name}_ids.parquet')
  del data
  print(f"Archivo '{name}_ids.parquet' guardado: {time.time() - t0:.2f} s.")

Archivos auxiliares guardados: 190.71 s.
Archivo 'tomas_beneficiario_ids.parquet' guardado: 201.30 s.
Archivo 'tomas_normalizado_ids.parquet' guardado: 256.39 s.
Archivo 'sociodemo_ids.parquet' guardado: 294.09 s.
